In [ ]:
#OPENCV VERSION FOR USING SIFT
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0MB 146kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 144kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
#for installing rar file
!pip install pyunpack
!pip install patool

     |████████████████████████████████| 81kB 7.4MB/s 


In [ ]:
!git clone https://github.com/vkhoi/KTH-Action-Recognition.git

Cloning into 'KTH-Action-Recognition'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 709.16 KiB | 18.66 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
%cd /content/KTH-Action-Recognition/dataset

/content/KTH-Action-Recognition/dataset


In [ ]:
from pyunpack import Archive
Archive('boxing.rar').extractall('/content/KTH-Action-Recognition/dataset')
Archive('handclapping.rar').extractall('/content/KTH-Action-Recognition/dataset')
Archive('handwaving.rar').extractall('/content/KTH-Action-Recognition/dataset')
Archive('jogging.rar').extractall('/content/KTH-Action-Recognition/dataset')
Archive('running.rar').extractall('/content/KTH-Action-Recognition/dataset')
Archive('walking.rar').extractall('/content/KTH-Action-Recognition/dataset')

!rm *.rar

In [ ]:
#CODE FOR EXTRACTING Spatio temporal SIFT/BEBLID VALUES as features and storing as feature files all categories in one pickle file

import cv2
import numpy as np
import os
import pickle
import cv2
import numpy as np

import math
from sklearn.cluster import KMeans
from sklearn import preprocessing

def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    #sift = cv2.xfeatures2d.BEBLID_create(1)
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    descriptors = sift.compute(img, keypoints)[1]
    
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors


CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", "walking"]

if __name__ == "__main__":

    # Create directory to store extracted pixel features.
    os.makedirs("data", exist_ok=True)

    
    n_processed_files = 0
    features = []
    L = 3

    for category in CATEGORIES:
        print("Processing category %s" % category)

        # Get all files in current category's folder.
        folder_path = os.path.join("..", "dataset", category)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
            filepath = os.path.join("..", "dataset", category, filename)
            path = "/content/KTH-Action-Recognition/dataset/" + category + "/" + filename
            vid = cv2.VideoCapture(filepath)

            features_current_file_new = []
            count_frame = 0
            while vid.isOpened():
              ret, frame = vid.read()
              count_frame = count_frame + 1
              if not ret:
                break
              if(count_frame < (2**L + 1)):
  # Only care about gray scale.
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                W = frame.shape[1]
                H = frame.shape[0]   
                h = []
                for l in range(L+1):
                  w_step = math.floor(W/(2**l))
                  h_step = math.floor(H/(2**l))
                  x, y = 0, 0
                  for i in range(1,2**l + 1):
                    x = 0
                    for j in range(1, 2**l + 1):                
                      desc = extract_denseSIFT(frame[y:y+h_step, x:x+w_step])                
                #print("type:",desc is None, "x:",x,"y:",y, "desc_size:",desc is None)
                      kmeans = KMeans(n_clusters=60, init='k-means++', max_iter=300, n_init=10, random_state=0)
                      predict = kmeans.fit_predict(desc)
                      histo = np.bincount(predict, minlength=60).reshape(1,-1).ravel()
                      weight = 2**(l-L)
                      h.append(weight*histo)
                      x = x + w_step
                    y = y + h_step
            
                hist = np.array(h).ravel()
    # normalize hist
                dev = np.std(hist)
                hist -= np.mean(hist)
                hist /= dev

                features_current_file_new.append(hist)
    
                features_current_file = np.array(features_current_file_new)


            # Store features in current file.
    

# No of frames
#print((features_current_file))  
 #     features_current_file = np.array(features_current_file_new)

            print(filename)    
            print(path)
            features.append({
                "filename": filename,
                "path" : path,
                "category": category,
                "features": features_current_file 
            })

            n_processed_files += 1
            if n_processed_files % 30 == 0:
                print("Done %d files" % n_processed_files)

        # Dump data to file.
pickle.dump(features, open("data/SIFT_stpm_next24_KTH.p" , "wb"))


Processing category boxing
person04_boxing_d4_uncomp.avi
/content/KTH-Action-Recognition/dataset/boxing/person04_boxing_d4_uncomp.avi
person03_boxing_d4_uncomp.avi
/content/KTH-Action-Recognition/dataset/boxing/person03_boxing_d4_uncomp.avi
person04_boxing_d3_uncomp.avi
/content/KTH-Action-Recognition/dataset/boxing/person04_boxing_d3_uncomp.avi
person04_boxing_d1_uncomp.avi
/content/KTH-Action-Recognition/dataset/boxing/person04_boxing_d1_uncomp.avi
Processing category handclapping
person04_handclapping_d2_uncomp.avi
/content/KTH-Action-Recognition/dataset/handclapping/person04_handclapping_d2_uncomp.avi
person03_handclapping_d2_uncomp.avi
/content/KTH-Action-Recognition/dataset/handclapping/person03_handclapping_d2_uncomp.avi
person03_handclapping_d1_uncomp.avi
/content/KTH-Action-Recognition/dataset/handclapping/person03_handclapping_d1_uncomp.avi
person04_handclapping_d1_uncomp.avi
/content/KTH-Action-Recognition/dataset/handclapping/person04_handclapping_d1_uncomp.avi
Processing c

In [ ]:
file = open('/content/KTH-Action-Recognition/dataset/data/SIFT_stpm_next24_KTH.p', 'rb')


# dump information to that file
data = pickle.load(file)


# close the file
file.close()



print('Showing the pickled data:')


cnt = 0
for item in data:
    cnt = cnt + 1
    #print('The data ', cnt, ' is : ', item['path'])
    print('The data ', cnt, ' is : ', (item['features']))


Showing the pickled data:
The data  1  is :  [[ 5.2589436  10.63721166  4.74321927 ... -0.48769898 -0.48769898
  -0.48769898]
 [ 3.83811587  4.57085491  3.03210291 ... -0.48504451 -0.48504451
  -0.48504451]
 [ 3.65877818  3.65877818  4.16713959 ... -0.48073617 -0.48073617
  -0.48073617]
 ...
 [ 3.14656641 11.27040807 14.4619173  ... -0.48014863 -0.48014863
  -0.48014863]
 [ 2.27523959  8.14815557 14.38359723 ... -0.47995556 -0.47995556
  -0.47995556]
 [ 3.555155    0.98323651  5.24527286 ... -0.48643119  0.10143589
  -0.48643119]]
The data  2  is :  [[ 9.13389053  2.84706087  6.43953496 ... -0.4246566  -0.4246566
  -0.4246566 ]
 [ 5.15470798  7.43498069  0.8475262  ... -0.41929198 -0.41929198
  -0.41929198]
 [ 2.26503934  9.75533514  7.00249139 ... -0.42378479 -0.42378479
  -0.42378479]
 ...
 [ 1.61136656 15.49083888 16.81269338 ... -0.43750792 -0.43750792
  -0.43750792]
 [ 1.8901624  10.61489294  7.35144411 ... -0.44087248 -0.44087248
  -0.44087248]
 [ 1.12111844  4.60437678  2.411214

In [ ]:
#CODE FOR EXTRACTING Spatio temporal OPTICAL FLOW VALUES as features and storing as feature files all categories in one pickle file

import cv2
import numpy as np
import os
import pickle
import cv2
import numpy as np

import math
from sklearn.cluster import KMeans
from sklearn import preprocessing



CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", "walking"]

#TRAIN_PEOPLE_ID = [11, 12, 13, 14, 15, 16, 17, 18]
#DEV_PEOPLE_ID = [19, 20, 21, 23, 24, 25, 1, 4]
#TEST_PEOPLE_ID = [22, 2, 3, 5, 6, 7, 8, 9, 10]


if __name__ == "__main__":

    # Create directory to store extracted pixel features.
    os.makedirs("data", exist_ok=True)

    # Setup parameters for optical flow.
    farneback_params = dict(winsize = 20, iterations=1, flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN, levels=1, pyr_scale=0.5, poly_n=5, poly_sigma=1.1, flow=None)

    n_processed_files = 0
    features = []
    L = 3

    for category in CATEGORIES:
        print("Processing category %s" % category)

        # Get all files in current category's folder.
        folder_path = os.path.join("..", "dataset", category)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
            filepath = os.path.join("..", "dataset", category, filename)
            path = "/content/KTH-Action-Recognition/dataset/" + category + "/" + filename
            vid = cv2.VideoCapture(filepath)

            features_current_file_new = []
            count_frame = 0
            prev_frame = None
            while vid.isOpened():
              ret, frame = vid.read()
              count_frame = count_frame + 1
              if not ret:
                break
              # Only care about gray scale.
              if(count_frame < 2**L + 1):
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              #print(frame)
                if(prev_frame is not None):

                  W = frame.shape[1]
                  H = frame.shape[0]   
                  h = []
                  for l in range(L+1):
                      w_step = math.floor(W/(2**l))
                      h_step = math.floor(H/(2**l))
                      x, y = 0, 0
                      for i in range(1,2**l + 1):
                          x = 0
                          for j in range(1, 2**l + 1): 
                            #desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])               
                            flows = cv2.calcOpticalFlowFarneback(prev_frame[y:y+h_step, x:x+w_step], frame[y:y+h_step, x:x+w_step],**farneback_params)
                            feature = []
                            #for r in range(w_step):
                              #if r % 10 != 0:
                                #continue
                              #for c in range(h_step):
                                #if c % 10 != 0:
                                  #continue
                                #feature.append(flows[r,c,0])
                                #feature.append(flows[r,c,1])
                            feature.append(flows)
                            feature = np.array(feature)
                            feature = feature.reshape(-1, 1)

                            kmeans = KMeans(n_clusters=60, init='k-means++', max_iter=300, n_init=10, random_state=0)
                            predict = kmeans.fit_predict(feature)

                            histo = np.bincount(predict, minlength=60).reshape(1,-1).ravel()
                            weight = 2**(l-L)
                            h.append(weight*histo)
                            x = x + w_step
                          y = y + h_step
                        
                  hist = np.array(h).ravel()
                # normalize hist
                  dev = np.std(hist)
                  hist -= np.mean(hist)
                  hist /= dev
                  features_current_file_new.append(hist)
                #print(flows)
                  
                  #features_current_file_new.append(feature)
                prev_frame = frame
            features_current_file_new = np.array(features_current_file_new)
            #print(np.array(features_current_file_new))
            print((features_current_file_new.shape))


            # Store features in current file.
    

# No of frames
#print((features_current_file))  
 #     features_current_file = np.array(features_current_file_new)

            print(filename)    
            print(path)
            features.append({
                "filename": filename,
                "path" : path,
                "category": category,
                "features": features_current_file_new 
            })

            n_processed_files += 1
            if n_processed_files % 30 == 0:
                print("Done %d files" % n_processed_files)

        # Dump data to file.
pickle.dump(features, open("data/optflow_stpm.p" , "wb"))